# Scenario Testing BHC Module

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy import linalg as la
from scipy import random as rnd
from scipy.special import gamma
import matplotlib.pyplot as plt
from sklearn import datasets as data
from sklearn import metrics

# load our package
import bhc

## Metrics

In [ ]:
def purity_score(y_true, y_pred):
    """
    Inputs:
    y_true - an array of the true cluster assignments of type int
    y_pred - an array of the predicted cluster assignments of type int
    
    Output:
    Cluster purity metric (range [0,1]) for the cluster assignments 
    relative to their true values.
    
    Reference:
    https://en.wikipedia.org/wiki/Cluster_analysis#Evaluation_and_assessment
    """
    
    from sklearn import metrics
    
    # build confusion matrix
    conf_mat = metrics.cluster.contingency_matrix(y_true, y_pred)
    #print(conf_mat)
    
    # calculate score
    score = np.sum(np.amax(conf_mat, axis=0))/np.sum(conf_mat)
    
    return np.round(score,2)

In [ ]:
def adj_purity_score(y_true, y_pred, n_clust_true):
    """
    Inputs:
    y_true - an array of the true cluster assignments of type int
    y_pred - an array of the predicted cluster assignments of type int
    n_clust_true - int, the true number of classes in the data 
    
    Output:
    Cluster purity metric (range [0,1]) for the cluster assignments 
    relative to their true values.
    
    Reference:
    https://en.wikipedia.org/wiki/Cluster_analysis#Evaluation_and_assessment
    """
    
    from sklearn import metrics
    
    # build confusion matrix
    conf_mat = metrics.cluster.contingency_matrix(y_true, y_pred)
    
    # find max number of points for each class
    class_max = np.amax(conf_mat, axis=0)
    class_max.sort()
    # calculate score
    score = np.sum(class_max[::-1][0:n_clust_true])/np.sum(conf_mat)
    
    return np.round(score,2)

In [ ]:
test= np.array([[1,2,3,4,5],[0,3,1,2,8]])
max_test = test.max(axis=0)
max_test.sort()
max_test[::-1][0:3]

## Three component Gaussian mixture (simulated data set):

In [ ]:
n_dim = 3 # dimensionality of problem

# bivariate gaussian params
mu1 = np.zeros(3)
cov1 = np.eye(3)

mu2 = np.array([5, 3, 0])
cov2 = np.eye(3)

mu3 = np.array([8, 12, 1])
cov3 = np.eye(3)* 0.5

# multinom params
p1 = 0.3
p2 = 0.4
p3 = 1 - p2 - p1

# number of total draws
draws = 100

In [ ]:
# random draws
rnd.seed(1)

knum = rnd.multinomial(draws, (p1, p2, p3))

gaus1 = rnd.multivariate_normal(mu1, cov1, knum[0])
gaus2 = rnd.multivariate_normal(mu2, cov2, knum[1])
gaus3 = rnd.multivariate_normal(mu3, cov3, knum[2])

# join columns into dataframe
x1 = pd.Series(np.r_[gaus1[:, 0], gaus2[:, 0], gaus3[:, 0]])
x2 = pd.Series(np.r_[gaus1[:, 1], gaus2[:, 1], gaus3[:, 1]])
x3 = pd.Series(np.r_[gaus1[:, 2], gaus2[:, 2], gaus3[:, 2]])
c = pd.Series(np.r_[np.zeros(knum[0]), np.ones(knum[1]), np.ones(knum[2]) * 2])
dat = {"x1" : x1, "x2" : x2, "x3" : x3, "c" : c}
clustData = pd.DataFrame(dat)

In [ ]:
plt.scatter(clustData["x1"], clustData["x2"], c = clustData["c"])
plt.title("Visualizing the Clusters")
plt.show()

### Using BHC

In [ ]:
# standardize data
clustData_std = (clustData.values[:, :3] - 
                 clustData.values[:, :3].mean(axis = 0))/(clustData.values[:, :3].std(axis=0))
clustData_std.mean(axis=0)

In [ ]:
# priors distribution hyper-parameters
gcPriors = {
    "clusterConcentrationPrior" : {"alpha" : 0.0001},
    "diffuseInvWishPrior" : {"df" : 3, "scale" : np.eye(3)}, # inv wishart params
    "diffuseNormPrior" : {"loc" : clustData.values[:, :3].mean(axis=0),
                          "scale" : np.eye(3),
                          "meanscale" : 1}, # mvtnormal params
}

In [ ]:
gc = clustData.values[:, :3] #clustData_std
gctree = bhc.HierarchyTree(X = gc, allParams = gcPriors)

In [ ]:
gctree.grow_tree()

In [ ]:
# pre prunning tree summary
gctree.tree_summary()

In [ ]:
gctree.prune_tree()

In [ ]:
gctree.tree_summary()

In [ ]:
gctree.generate_clust_frame()

In [ ]:
plt.scatter("Dim_0", "Dim_1", c = "clustnum", data=gctree.clustDF)
plt.show()

In [ ]:
purity_score(clustData["c"], gctree.clustDF["clustnum"])

In [ ]:
adj_purity_score(clustData["c"], gctree.clustDF["clustnum"], n_dim)

### Using sklearn hierarchical clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
gc_tree_cv = []
for n_clust in range(1,10):
    gc_tree1 = AgglomerativeClustering(n_clusters = n_clust)
    gc_tree1.fit(gc)
    gc_tree_cv.append((n_clust, 
                       purity_score(clustData["c"], gc_tree1.labels_),
                       adj_purity_score(clustData["c"], gc_tree1.labels_, n_dim)
                      ))

In [ ]:
gc_tree_cv

### Using k-means clustering

In [ ]:
from sklearn.cluster import k_means

In [ ]:
gc_kmeans_cv = []
for n_clust in range(1,10):
    centr, km_labs, inert = k_means(gc, n_clusters = n_clust)
    gc_kmeans_cv.append((n_clust, 
                         purity_score(clustData["c"], km_labs),
                         adj_purity_score(clustData["c"], km_labs, n_dim)
                        ))

In [ ]:
gc_kmeans_cv

## Computational Speed

In [ ]:
%%timeit -r3 -n3
gctree = bhc.HierarchyTree(X = gc, allParams = gcPriors)
gctree.grow_tree()

In [ ]:
%%timeit -r3 -n3
gc_tree1 = AgglomerativeClustering(n_clusters = 10)
gc_tree1.fit(gc)

In [ ]:
%%timeit -r3 -n3
centr, km_labs, inert = k_means(gc, n_clusters = 10)

## Iris data

In [ ]:
iris = data.load_iris()

In [ ]:
iris.data.shape

In [ ]:
n_dim_iris = np.unique(iris.target).shape[0]
n_dim_iris

In [ ]:
np.unique(iris.target)

In [ ]:
# standardize data
iris_data_std = (iris.data - iris.data.mean(axis=0))/(iris.data.std(axis=0))

### Using BHC

In [ ]:
k = iris.data.shape[1]
# priors distribution hyper-parameters
irisPriors = {
    "clusterConcentrationPrior" : {"alpha" : 0.0001},
    "diffuseInvWishPrior" : {"df" : 4, "scale" : np.eye(k)}, # inv wishart params
    "diffuseNormPrior" : {"loc" : iris_data_std.mean(axis = 0),
                          "scale" : np.eye(k),
                          "meanscale" : 1}, # mvtnormal params
}

In [ ]:
iris_tree = bhc.HierarchyTree(X = iris_data_std, allParams = irisPriors)

In [ ]:
iris_tree.grow_tree()

In [ ]:
iris_tree.tree_summary()

In [ ]:
iris_tree.prune_tree()

In [ ]:
iris_tree.tree_summary()

In [ ]:
iris_tree.generate_clust_frame()
purity_score(iris.target, iris_tree.clustDF["clustnum"])

In [ ]:
adj_purity_score(iris.target, iris_tree.clustDF["clustnum"], n_dim_iris)

### Using sklearn hierarchical clustering

In [ ]:
iris_tree1_cv = []
for n_clust in range(1,10):
    iris_tree1 = AgglomerativeClustering(n_clusters = n_clust)
    iris_tree1.fit(iris_data_std)
    iris_tree1_cv.append((n_clust, 
                       purity_score(iris.target, iris_tree1.labels_),
                       adj_purity_score(iris.target, iris_tree1.labels_, n_dim_iris)
                      ))

In [ ]:
iris_tree1_cv

### Using k-means clustering

In [ ]:
iris_kmeans_cv = []
for n_clust in range(1,10):
    centr, km_labs, inert = k_means(iris_data_std, n_clusters = n_clust)
    iris_kmeans_cv.append((n_clust, 
                         purity_score(iris.target, km_labs),
                         adj_purity_score(iris.target, km_labs, n_dim_iris)
                        ))

In [ ]:
iris_kmeans_cv

## Wine data

In [ ]:
wine = data.load_wine()

In [ ]:
wine.data.shape

In [ ]:
n_dim_wine = np.unique(wine.target).shape[0]
n_dim_wine

### Using BHC

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [ ]:
# standardize data
wine_data_std = (wine.data - wine.data.mean(axis=0))/(wine.data.std(axis=0))

In [ ]:
wine_pca = PCA(n_components = 3)
wine_red = wine_pca.fit_transform(wine_data_std, wine.target)

In [ ]:
wine_red.shape

In [ ]:
wine_pca.explained_variance_ratio_

In [ ]:
wine_train, wine_test, target_train, target_test = train_test_split(wine_red, wine.target, test_size= 0.1)

In [ ]:
k = wine_red.shape[1]
# priors distribution hyper-parameters
winePriors = {
    "clusterConcentrationPrior" : {"alpha" : 0.0001},
    "diffuseInvWishPrior" : {"df" : k, "scale" : np.eye(k)}, # inv wishart params
    "diffuseNormPrior" : {"loc" : wine_red.mean(axis = 0),
                          "scale" : np.eye(k),
                          "meanscale" : 1}, # mvtnormal params
}

In [ ]:
wine_tree = bhc.HierarchyTree(X = wine_train, allParams = winePriors)

In [ ]:
wine_tree.grow_tree()

In [ ]:
wine_tree.tree_summary()

In [ ]:
wine_tree.prune_tree()

In [ ]:
wine_tree.generate_clust_frame()
purity_score(target_train, wine_tree.clustDF["clustnum"])

In [ ]:
adj_purity_score(target_train, wine_tree.clustDF["clustnum"], n_dim_wine)

### Using sklearn hierarchical clustering

In [ ]:
wine_tree1_cv = []
for n_clust in range(1,10):
    wine_tree1 = AgglomerativeClustering(n_clusters = n_clust)
    wine_tree1.fit(wine_train)
    wine_tree1_cv.append((n_clust, 
                       purity_score(target_train, wine_tree1.labels_),
                       adj_purity_score(target_train, wine_tree1.labels_, n_dim_wine)
                      ))

In [ ]:
wine_tree1_cv

### Using k-means clustering

In [ ]:
wine_kmeans_cv = []
for n_clust in range(1,10):
    centr, km_labs, inert = k_means(wine_train, n_clusters = n_clust)
    wine_kmeans_cv.append((n_clust, 
                         purity_score(target_train, km_labs),
                         adj_purity_score(target_train, km_labs, n_dim_wine)
                        ))

In [ ]:
wine_kmeans_cv

## Testing binomial

In [ ]:
n_dim = 3 # dimensionality of problem

# bernoulii params
mu1 = 0
mu2 = 0.5
mu3 = 1

# multinom params
p1 = 0.3
p2 = 0.4
p3 = 1 - p2 - p1

# number of total draws
draws = 100

# random draws
rnd.seed(1)

knum = rnd.multinomial(draws, (p1, p2, p3))

bern1 = rnd.binomial(1, mu1, knum[0])
bern2 = rnd.binomial(1, mu2, knum[1])
bern3 = rnd.binomial(1, mu2, knum[2])

# join columns into dataframe
x1 = pd.Series(np.r_[bern1, bern2, bern3])
x2 = pd.Series(np.r_[bern1, bern2, bern3])
c = pd.Series(np.r_[np.zeros(knum[0]), np.ones(knum[1]), np.ones(knum[2]) * 2])
dat = {"x1" : x1, "x2" : x2, "c" : c}
bernData = pd.DataFrame(dat)

In [ ]:
# priors distribution hyper-parameters
bernPriors = {
    "clusterConcentrationPrior" : {"alpha" : 0.0001},
    "alphaPrior" : {"succ" : 1},
    "betaPrior" : {"fail" : 1}
}

In [ ]:
bern_train = bernData.values[:, :2]
bern_tree = bhc.HierarchyTree(X = bern_train, allParams = bernPriors, family="beta-bern")

In [ ]:
bern_tree.grow_tree()

In [ ]:
bern_tree.tree_summary()

In [ ]:
bern_tree.prune_tree()

In [ ]:
bern_tree.tree_summary()

In [ ]:
bern_tree.generate_clust_frame()
purity_score(bernData["c"], bern_tree.clustDF["clustnum"])

In [ ]:
adj_purity_score(bernData["c"], bern_tree.clustDF["clustnum"],3)

In [ ]:
metrics.cluster.contingency_matrix(bernData["c"], bern_tree.clustDF["clustnum"])

In [ ]:
bern_tree1_cv = []
for n_clust in range(1,10):
    bern_tree1 = AgglomerativeClustering(n_clusters = n_clust)
    bern_tree1.fit(bern_train)
    bern_tree1_cv.append((n_clust, 
                       purity_score(bernData["c"], bern_tree.clustDF["clustnum"]),
                       adj_purity_score(bernData["c"], bern_tree.clustDF["clustnum"], 3)
                      ))

In [ ]:
bern_tree1_cv

In [ ]:
gc_kmeans_cv = []
for n_clust in range(1,10):
    centr, km_labs, inert = k_means(gc, n_clusters = n_clust)
    gc_kmeans_cv.append((n_clust, 
                         purity_score(clustData["c"], km_labs),
                         adj_purity_score(clustData["c"], km_labs, n_dim)
                        ))